In [6]:
# Desafio 1: Consolidação de arquivos CSV

# 1. Importando as bibliotecas necessárias
import pandas as pd  # Para manipulação de dados em tabelas

# 2. Carregando os arquivos CSV
# Lendo o arquivo de produtos
products_df = pd.read_csv('products.csv')
# Lendo o arquivo de pedidos
orders_df = pd.read_csv('orders.csv')

# 3. Realizando o Merge entre as tabelas
# Unimos os dados usando a coluna 'product_id' de orders e 'id' de products
merged_df = pd.merge(
    orders_df,         # DataFrame de pedidos
    products_df,       # DataFrame de produtos
    left_on='product_id',  # Coluna de referência em orders
    right_on='id'         # Coluna de referência em products
)

# 4. Calculando o preço total
# Criamos uma nova coluna chamada 'total_price'
merged_df['total_price'] = merged_df['quantity'] * merged_df['price']

# 5. Selecionando e renomeando as colunas necessárias
# Escolhemos apenas as colunas importantes para o arquivo final
final_df = merged_df[[
    'created_date',   # Data do pedido
    'id_x',           # ID do pedido (de orders)
    'name',           # Nome do produto (de products)
    'quantity',       # Quantidade do pedido
    'total_price'     # Preço total calculado
]]

# Renomeamos as colunas para o formato desejado
final_df.columns = [
    'order_created_date',
    'order_id',
    'product_name',
    'quantity',
    'total_price'
]

# 6. Salvando o resultado em um novo arquivo CSV
final_df.to_csv('order_full_information.csv', index=False)

print("Arquivo 'order_full_information.csv' criado com sucesso!")


Arquivo 'order_full_information.csv' criado com sucesso!


In [5]:
import pandas as pd

result = pd.read_csv('order_full_information.csv')

print(result)

   order_created_date  order_id product_name  quantity  total_price
0        '2024-12-01'         1   Product_11         1        69.06
1        '2024-12-01'         2   Product_17         2       197.02
2        '2024-12-01'         3   Product_19         1        51.94
3        '2024-12-01'         4   Product_12         1        50.99
4        '2024-12-01'         5   Product_11         2       138.12
5        '2024-12-01'         6   Product_17         1        98.51
6        '2024-12-01'         7    Product_5         5       222.95
7        '2024-12-01'         8    Product_5         4       178.36
8        '2024-12-02'         9    Product_5         2        89.18
9        '2024-12-02'        10    Product_1         4       370.20
10       '2024-12-02'        11   Product_15         5       488.60
11       '2024-12-02'        12    Product_7         4       399.84
12       '2024-12-02'        13   Product_13         3        46.59
13       '2024-12-03'        14   Product_13    

Desafio 2


In [1]:
# 🚀 Desafio 2: Conversão de Moeda e KPIs

import pandas as pd
import requests

# ✅ Parte 1: Conversão de Moeda

# 1. Carregar o arquivo consolidado
try:
    df = pd.read_csv('order_full_information.csv')
    print("✅ Arquivo 'order_full_information.csv' carregado com sucesso!")
except FileNotFoundError:
    print("❌ Erro: Arquivo 'order_full_information.csv' não encontrado.")
    exit()
except Exception as e:
    print(f"❌ Erro ao carregar arquivo: {e}")
    exit()

# 2. Obter a taxa de câmbio BRL -> USD
API_KEY = 'SUA_API_KEY'  # Substitua pela sua chave da FreeCurrencyAPI
API_URL = f'https://api.freecurrencyapi.com/v1/latest?apikey={API_KEY}&currencies=USD&base_currency=BRL'

try:
    response = requests.get(API_URL)
    response.raise_for_status()  # Verifica erros na requisição
    data = response.json()
    exchange_rate = float(data['data']['USD'])
    print(f"✅ Taxa de câmbio BRL → USD: {exchange_rate}")
except KeyError:
    print("❌ Erro: Chave inesperada nos dados da API.")
    exit()
except requests.exceptions.RequestException as e:
    print(f"❌ Erro de conexão com a API: {e}")
    exit()

# 3. Adicionar colunas para total_price_br e total_price_us
try:
    df['total_price_br'] = df['total_price']
    df['total_price_us'] = df['total_price'] * exchange_rate
    print("✅ Colunas 'total_price_br' e 'total_price_us' adicionadas com sucesso!")
except KeyError:
    print("❌ Erro: Coluna 'total_price' não encontrada no arquivo.")
    exit()
except Exception as e:
    print(f"❌ Erro ao adicionar colunas: {e}")
    exit()

# 4. Salvar os resultados em um novo arquivo
try:
    df.to_csv('fixed_order_full_information.csv', index=False)
    print("✅ Arquivo 'fixed_order_full_information.csv' criado com sucesso!")
except Exception as e:
    print(f"❌ Erro ao salvar arquivo: {e}")
    exit()

# ✅ Parte 2: Cálculo de KPIs

# 1. Data com maior número de pedidos
try:
    max_orders_date = df['order_created_date'].value_counts().idxmax()
except KeyError:
    print("❌ Erro: Coluna 'order_created_date' não encontrada.")
    max_orders_date = 'N/A'

# 2. Produto mais demandado e total vendido
try:
    most_demanded_product = df.groupby('product_name')['quantity'].sum().idxmax()
    most_demanded_product_total = df.groupby('product_name')['total_price_br'].sum().loc[most_demanded_product]
except KeyError:
    print("❌ Erro: Coluna 'product_name' ou 'quantity' não encontrada.")
    most_demanded_product = 'N/A'
    most_demanded_product_total = 0.0

# 3. Top 3 categorias mais demandadas
try:
    if 'category' in df.columns:
        top_categories = df.groupby('category')['quantity'].sum().nlargest(3)
        top_categories_list = ', '.join(top_categories.index)
    else:
        top_categories_list = 'Categoria não disponível'
except Exception as e:
    print(f"❌ Erro ao calcular categorias: {e}")
    top_categories_list = 'Erro ao calcular categorias'

# 4. Salvar KPIs em um arquivo CSV
kpi_data = {
    'kpi_name': [
        'Max Orders Date', 
        'Most Demanded Product', 
        'Most Demanded Product Total Sales (BRL)', 
        'Top 3 Categories'
    ],
    'kpi_value': [
        max_orders_date, 
        most_demanded_product, 
        most_demanded_product_total, 
        top_categories_list
    ]
}

try:
    kpi_df = pd.DataFrame(kpi_data)
    kpi_df.to_csv('kpi_product_orders.csv', index=False)
    print("✅ Arquivo 'kpi_product_orders.csv' criado com sucesso!")
except Exception as e:
    print(f"❌ Erro ao salvar KPIs: {e}")
    exit()


ModuleNotFoundError: No module named 'requests'